In [1]:
import pandas as pd

all_skills = 'skills/Skills.xlsx'
digital_skills = 'skills/Digital Skills.xlsx'

In [2]:
def extract_skills(file_name):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    all_skills = []
    for key, df in dfs.items():
        all_skills.extend(df['Skill'].unique().tolist())
    df_skills = pd.DataFrame({'skill': all_skills})
    return df_skills.drop_duplicates()

def parent_check(file_name, df_skills):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    for key, df in dfs.items():
        df_error = df.loc[~df['Parent'].isin(df_skills['skill'])]
        print(df_error['Parent'].unique())

def extract_relation(file_name, df_skills):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    df_rel = pd.DataFrame(columns=['Skill', 'Parent'])
    skill_to_id_dict = df_skills.set_index('skill').to_dict()['skill_id']
    for key, df in dfs.items():
        df_rel = df_rel.append(df.replace(skill_to_id_dict))
    df_rel = df_rel.reset_index(drop=True)
    df_rel = df_rel.dropna()
    df_rel = df_rel.drop_duplicates()
    df_rel['Parent'] = df_rel['Parent'].astype(int)
    return df_rel

### Extract Skills

In [3]:
# All skills
df_s1 = extract_skills(all_skills)
df_s1.head()

,skill
0,Applied Science
1,Arts
2,Business
3,Computer Science
4,Data Science


In [4]:
df_s1.describe(include='object')

,skill
count,806
unique,806
top,Convex Geometry
freq,1


In [5]:
# Digital skills
df_s2 = extract_skills(digital_skills)
df_s2.head()

,skill
0,Data Analysis
1,Machine Learning
2,Algorithm
3,Artificial Intelligence
4,Big Data


In [6]:
df_s2.describe(include='object')

,skill
count,594
unique,594
top,Convolutional Neural Network (CNN)
freq,1


In [7]:
# Combine
df_skills = df_s1.append(df_s2)
df_skills = df_skills.drop_duplicates()
df_skills['skill_id'] = df_skills.groupby('skill').grouper.group_info[0]
df_skills = df_skills.reset_index(drop=True)
df_skills = df_skills[['skill_id', 'skill']]
df_skills.describe(include='object')

,skill
count,1336
unique,1336
top,Convex Geometry
freq,1


### Data Checking/Cleaning

In [8]:
parent_check(all_skills, df_skills)

[nan]
[]
[]
[]
[]


In [9]:
parent_check(digital_skills, df_skills)

[nan]
[]
[]
[]
[]


### Extract Relation

In [10]:
df_r1 = extract_relation(all_skills, df_skills)
df_r2 = extract_relation(digital_skills, df_skills)
# Combine
df_rel = df_r1.append(df_r2)
df_rel = df_rel.drop_duplicates()
df_rel = df_rel.reset_index(drop=True)
df_rel.describe(include='all')

,Skill,Parent
count,1491.0,1491.000000
unique,1327.0,NaN
top,744.0,NaN
freq,5.0,NaN
mean,NaN,611.616365
std,NaN,387.119012
min,NaN,3.000000
25%,NaN,259.000000
50%,NaN,619.000000
75%,NaN,977.000000


### Export

In [11]:
df_skills.to_csv('skills_db/skill.csv')
df_rel.to_csv('skills_db/skill_tree.csv')